In [ ]:
! pip install groq datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Succ

# INSTRUCTION TUNING DATASET

In [ ]:
import os
from groq import Groq
from datasets import Dataset, DatasetDict, load_dataset
import json

topic = "Machine Learning"
n_subtopics = 4
n_questions = 10


from groq import Groq

client = Groq(
    api_key="",
)

In [ ]:

# 1. Subtopics Generation

TOPIC_GENERATION_PROMPT_TEMPLATE = """\
Given a topic, generate a list of {n_subtopics} subtopics that are related to the topic with topics as key.

Topic: {topic}

The list must be without numbers, and without any description of the subtopics. The subtopics should be separated by a comma. There must be no other text than the list.

Return a JSON object with the list of subtopics separated by commas and a space.
"""

def generate_subtopics(client, topic, n_subtopics):
    prompt = TOPIC_GENERATION_PROMPT_TEMPLATE.format(topic=topic, n_subtopics=n_subtopics)
    response = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=[
            {"role": "user",
             "content": prompt}
        ],
        temperature=0,
        max_tokens=1024,
        response_format={"type": "json_object"}
    )
    return response

responses = generate_subtopics(client, topic=topic, n_subtopics=n_subtopics)
print(responses)
print(responses.choices[0].message.content)
response_json = json.loads(responses.choices[0].message.content)
subtopic_list = list(response_json.values())[0]
subtopic_list = subtopic_list.split(",")
print(subtopic_list)

ChatCompletion(id='chatcmpl-19a7039b-c434-423f-b5a5-5bc634e3f567', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "Machine Learning": "Supervised Learning, Unsupervised Learning, Reinforcement Learning, Deep Learning"\n}', role='assistant', function_call=None, tool_calls=None))], created=1722941714, model='llama-3.1-70b-versatile', object='chat.completion', system_fingerprint='fp_9260b4bb2e', usage=CompletionUsage(completion_tokens=25, prompt_tokens=100, total_tokens=125, completion_time=0.1, prompt_time=0.023072095, queue_time=None, total_time=0.123072095), x_groq={'id': 'req_01j4kn8bbafwnb88tvzmvm198z'})
{
  "Machine Learning": "Supervised Learning, Unsupervised Learning, Reinforcement Learning, Deep Learning"
}
['Supervised Learning', ' Unsupervised Learning', ' Reinforcement Learning', ' Deep Learning']


In [ ]:
QUESTION_PROMPT_TEMPLATE = """\
Given a topic, generate {n_questions} questions that could be asked about that topic. Your response should be in a list format.

Topic: {sub_topic}

The list must be without numbers. The questions should be separated by a comma. There must be no other text than the list no [].

Return a JSON object with the list of questions separated by commas and a space.
"""
def generate_questions(client, sub_topic, n_questions):
    prompt = QUESTION_PROMPT_TEMPLATE.format(sub_topic=sub_topic, n_questions=n_questions)
    response = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=[
            {"role": "user",
             "content": prompt}
        ],
        temperature=0,
        max_tokens=1024,
        response_format={"type": "json_object"}
    )
    return response

def question_generator(client, subtopic_list, n_question):
    tasks = [generate_questions(client, subtopic, n_question) for subtopic in subtopic_list]
    question_list = tasks
    return question_list

question_lists = question_generator(client, subtopic_list, n_questions)
all_questions = []
for responses in question_lists:
    print(responses.choices[0].message.content)
    response_json = json.loads(responses.choices[0].message.content)
    question_list = list(response_json.values())[0]
    question_list = question_list.split(",")
    print(question_list)
    all_questions.extend(question_list)
print(all_questions)

{"questions": "What is supervised learning, How does supervised learning differ from unsupervised learning, What are the key components of a supervised learning algorithm, What is the role of labeled data in supervised learning, How do you handle missing or noisy data in supervised learning, What are some common supervised learning algorithms, How do you evaluate the performance of a supervised learning model, What is overfitting in supervised learning, How do you prevent overfitting in supervised learning, What are some real-world applications of supervised learning"}
['What is supervised learning', ' How does supervised learning differ from unsupervised learning', ' What are the key components of a supervised learning algorithm', ' What is the role of labeled data in supervised learning', ' How do you handle missing or noisy data in supervised learning', ' What are some common supervised learning algorithms', ' How do you evaluate the performance of a supervised learning model', ' Wh

In [ ]:
print(len(all_questions))

40


In [ ]:
# 3. Responses Generation

from tqdm import tqdm

RESPONSE_PROMPT_TEMPLATE = """\
Given a question, generate an answer which should be less than 100 words that could be given to that question.

Question: {question}

Your Answer:
"""
def generate_responses(client, question):
    prompt = RESPONSE_PROMPT_TEMPLATE.format(question=question)
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "user",
             "content": prompt}
        ],
        temperature=0,
        top_p=0.7,
        max_tokens=1024
    )
    return response

def response_generator(client, question_list):
    tasks = [generate_responses(client, question) for question in all_questions]
    response_list = tasks
    return response_list

answer_list = response_generator(client, all_questions)
answer_list = [response.choices[0].message.content for response in tqdm(answer_list)]
answer_list

100%|██████████| 40/40 [00:00<00:00, 156358.02it/s]


['Supervised learning is a type of machine learning where an algorithm is trained on labeled data to learn a mapping between inputs and outputs. The algorithm is shown examples of input data and their corresponding correct outputs, allowing it to learn patterns and relationships in the data. The goal is to enable the algorithm to make accurate predictions on new, unseen data. Supervised learning is commonly used in applications such as image classification, natural language processing, and regression analysis, where the algorithm is trained to predict a specific output based on input data.',
 'Supervised learning and unsupervised learning are two main types of machine learning approaches. The key difference between them lies in the type of data used for training.\n\nIn supervised learning, the model is trained on labeled data, where the correct output is already known. The model learns to map inputs to outputs based on the labeled data, enabling it to make predictions on new, unseen da

In [ ]:
qa_pair_list = []
for question, answer in zip(all_questions, answer_list):
    qa_pair_list.append(
        {
            "question": question,
            "answer": answer
        }
    )

In [ ]:
qa_pair_list[0]

{'question': 'What is supervised learning',
 'answer': 'Supervised learning is a type of machine learning where an algorithm is trained on labeled data to learn a mapping between inputs and outputs. The algorithm is shown examples of input data and their corresponding correct outputs, allowing it to learn patterns and relationships in the data. The goal is to enable the algorithm to make accurate predictions on new, unseen data. Supervised learning is commonly used in applications such as image classification, natural language processing, and regression analysis, where the algorithm is trained to predict a specific output based on input data.'}

In [ ]:
with open('synthetic_data.jsonl', 'w') as f:
    for item in qa_pair_list:
        f.write(json.dumps(item))
        f.write('\n')

In [ ]:
print(len(qa_pair_list))

40


In [ ]:
dataset = Dataset.from_list(qa_pair_list)
dataset_dict = DatasetDict({"train": dataset})
dataset_dict.push_to_hub("Vasanth/synthetic-llama-datagen")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Vasanth/synthetic-llama-datagen/commit/da2a61537e7811663b38a68937d995c211d249c1', commit_message='Upload dataset', commit_description='', oid='da2a61537e7811663b38a68937d995c211d249c1', pr_url=None, pr_revision=None, pr_num=None)

# PREFERENCE DATASET

In [ ]:
import json
from datasets import load_dataset
import groq
from rich import print
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv())

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
os.environ["GROQ_API_KEY"] = os.getenv("GROQQ_API_KEY")

# Load a real dataset from Hugging Face
dataset = load_dataset("squad_v2", split="train")

# Convert dataset to list of dictionaries
json_data = dataset.select(range(100)).to_dict()

print("Dataset Rows:", len(json_data['id']))
# print({key: json_data[key][0] for key in json_data})

# Function to format the input for the model
def format_input(context, question):
    return (
        "### Context:\n" + context +
        ("\n\n### Question:\n" + question if question else "")
    )

# Initialize the Ollama client
client = groq.Groq()  # Replace with your actual API key if required

# Initialize new keys in the json_data dictionary
json_data['rejected'] = [''] * len(json_data['id'])
json_data['chosen'] = [''] * len(json_data['id'])

# Process each entry in the dataset
for i in range(len(json_data['id'])):
    context = json_data['context'][i]
    question = json_data['question'][i]
    answer = json_data['answers'][i]['text'][0] if json_data['answers'][i]['text'] else "No answer"

    print("Rejected Answer:", answer)

    prompt_text = format_input(context, question)

    prompt = (
        f"Rewrite `{prompt_text}` output to be concise and clear: {answer}. "
        "Ensure the response is easy to understand, professional and as a full sentense. Just respond only with the Answer"
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": ""
            },
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama-3.1-8b-instant",
    )

    response = chat_completion.choices[0].message.content
    chosen_answer = response
    print("Chosen Answer:", chosen_answer)

    json_data['rejected'][i] = answer
    json_data['chosen'][i] = chosen_answer

# Convert back to dictionary format expected by json.dump
final_data = [{key: json_data[key][i] for key in json_data} for i in range(len(json_data['id']))]

with open("preference_dataset.json", "w") as file:
    json.dump(final_data, file, indent=4)

new_data = load_dataset("json", data_files=["preference_dataset.json"])
new_data.push_to_hub("Vasanth/squad-demo-preference-dataset")